In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import os
import cv2
import math
import numpy as np
from skimage import measure
from skimage.measure import regionprops
from skimage.morphology import opening, erosion, dilation, disk
import matplotlib.pyplot as plt

In [0]:
TRAIN = ['232', '263', '230', '242']
TEST  = [ '239','221', '229', '246', '252']

In [0]:
# TRAIN_PATH = "/content/drive/My Drive/Breast Cancer Treatment/Numpy Arrays/Predicted Output/ER/UNET/4 Class/LadderNet/Train/"
# TEST_PATH = "/content/drive/My Drive/Breast Cancer Treatment/Numpy Arrays/Predicted Output/ER/UNET/4 Class/LadderNet/Test/"

TRAIN_PATH = "/content/drive/My Drive/Breast Cancer Treatment/Numpy Arrays/Predicted Output/ER/UNET/4 Class/LadderNet_29MAY/Train/"
TEST_PATH = "/content/drive/My Drive/Breast Cancer Treatment/Numpy Arrays/Predicted Output/ER/UNET/4 Class/LadderNet_29MAY/Test/"

# TRAIN_PATH = "/content/drive/My Drive/Breast Cancer Treatment/Numpy Arrays/Predicted Output/ER/HScoreNet/Train/"
# TEST_PATH = "/content/drive/My Drive/Breast Cancer Treatment/Numpy Arrays/Predicted Output/ER/HScoreNet/Test/"

MASK_PATH = "/content/drive/My Drive/Breast Cancer Treatment/Numpy Arrays/4 Class ER/Masks/"

In [0]:
def updateConfusionMatrix(ConfusionMatrix, PredictedImage, GroundTruth, PatientId, axis ):
    # Opening operation
    selem = disk(15)
    # PredictedImage = erosion(PredictedImage, selem)
    GroundTruth = dilation(GroundTruth,selem)
    GroundTruthLabel = measure.label(GroundTruth)

    rows,cols = PredictedImage.shape

    #  0 is background
    for label in range(1, GroundTruthLabel.max()+1):
      masklabel = np.zeros([rows,cols])
      masklabel[GroundTruthLabel==label] = 1

      resultantimage = cv2.multiply(PredictedImage, masklabel)

      areaGT = np.sum(masklabel)
      areaSM = np.sum(resultantimage)

      if(areaSM > 0 ):
        ConfusionMatrix[axis][0] = ConfusionMatrix[axis][0] + 1       #TP
        PredictedImage[GroundTruthLabel==label] = 0
      else:
        ConfusionMatrix[axis][2] = ConfusionMatrix[axis][2] + 1       #FN
    
    predictedprops = regionprops(measure.label(PredictedImage))

    for i in range(len(predictedprops)):
      if(predictedprops[i].area >= 2000):
        ConfusionMatrix[axis][1] += 1                                #FP

In [0]:
def getPatientWiseCount(ConfusionMatrix, PredictedPatient, GroundTruthPatient, PatientId):

  for imagenum in range(PredictedPatient.shape[0]):
  
    PredictedImage = PredictedPatient[imagenum]
    GroundTruth    = GroundTruthPatient[imagenum]
    rows,cols = PredictedImage.shape

    # dividing the image into strong,weak,intermediate
    predictedStrong = np.zeros([rows,cols])
    predictedWeak   = np.zeros([rows,cols])
    predictedIntermediate = np.zeros([rows,cols])

    gtStrong = np.zeros([rows,cols])
    gtWeak = np.zeros([rows,cols])
    gtIntermediate = np.zeros([rows,cols])

    predictedStrong[PredictedImage==255] = 1
    predictedIntermediate [PredictedImage==170] = 1
    predictedWeak[PredictedImage==85] = 1

    gtStrong[GroundTruth==255] = 1
    gtIntermediate[GroundTruth==170] = 1
    gtWeak[GroundTruth==85] = 1

    # 0th axis in confusion matrix is strong 
    # 1st axis in confusion matrix is intermediate 
    # 2nd axis in confusion matrix is weak
    updateConfusionMatrix(ConfusionMatrix, predictedStrong, gtStrong, PatientId, 0)
    updateConfusionMatrix(ConfusionMatrix, predictedIntermediate, gtIntermediate, PatientId, 1)
    updateConfusionMatrix(ConfusionMatrix, predictedWeak, gtWeak, PatientId, 2)

**SM = Segmented Mask**

**GT = Ground Truth** 

In [0]:
train_confusion_matrix = np.zeros([ 3, 3])
test_confusion_matrix  = np.zeros([ 3, 3])

for patientid in range(len(TRAIN)):
  patientSM = np.load(TRAIN_PATH + TRAIN[patientid] + '.npy')
  patientGT      = np.load(MASK_PATH + 'IHC ' + TRAIN[patientid] + '.npy')
  getPatientWiseCount(train_confusion_matrix, patientSM, patientGT, patientid)
  print(TRAIN[patientid] + ' done.')
  print(train_confusion_matrix)

232 done.
[[277.  14.  74.]
 [276. 161.  23.]
 [151.  96.  26.]]
263 done.
[[491.  72.  97.]
 [425. 302.  31.]
 [225. 176.  59.]]
230 done.
[[491.  72.  97.]
 [425. 302.  31.]
 [277. 178.  80.]]
242 done.
[[491.  72.  97.]
 [425. 302.  31.]
 [277. 178.  80.]]


In [0]:
for patientid in range(len(TEST)):
  patientSM = np.load(TEST_PATH + TEST[patientid] + '.npy')
  patientGT      = np.load(MASK_PATH + 'IHC ' + TEST[patientid] + '.npy')
  getPatientWiseCount(test_confusion_matrix, patientSM, patientGT, patientid)
  print(TEST[patientid] + ' done.')
  print(test_confusion_matrix)

239 done.
[[384. 185.   9.]
 [241.  65.  50.]
 [143. 133.  49.]]
221 done.
[[384. 185.   9.]
 [241.  65.  50.]
 [144. 133.  49.]]
229 done.
[[384. 185.   9.]
 [241.  65.  50.]
 [144. 133.  49.]]
246 done.
[[384. 185.   9.]
 [241.  65.  50.]
 [150. 133.  49.]]
252 done.
[[384. 185.   9.]
 [241.  66.  50.]
 [150. 134.  49.]]


In [0]:
def getMetrics(ConfusionMatrix, axis):
  precision = ConfusionMatrix[axis][0] / (ConfusionMatrix[axis][0] + ConfusionMatrix[axis][1] )
  recall    = ConfusionMatrix[axis][0] / (ConfusionMatrix[axis][0] + ConfusionMatrix[axis][2] )
  dice      = (2 * precision * recall) / (precision + recall+ 0.01)
  return precision, recall, dice

In [0]:
strain_precision, strain_recall, strain_dice = getMetrics(train_confusion_matrix, 0)
itrain_precision, itrain_recall, itrain_dice = getMetrics(train_confusion_matrix, 1)
wtrain_precision, wtrain_recall, wtrain_dice = getMetrics(train_confusion_matrix, 2)

print('TRAIN :\n\n')
print("CLASS\t\tPRECISION\t\tRECALL\t\t\tDICE")
print('Strong\t\t' + str(strain_precision) + '\t' + str(strain_recall) + '\t' + str(strain_dice))
print('Intermediate\t' + str(itrain_precision) +  '\t' + str(itrain_recall) + '\t' + str(itrain_dice))
print('Weak\t\t' + str(wtrain_precision) +   '\t' + str(wtrain_recall) + '\t' + str(wtrain_dice))

TRAIN :


CLASS		PRECISION		RECALL			DICE
Strong		0.872113676731794	0.8350340136054422	0.8482026186792665
Intermediate	0.5845942228335625	0.9320175438596491	0.7138056755583373
Weak		0.6087912087912087	0.7759103641456583	0.677374167589391


In [0]:
stest_precision, stest_recall, stest_dice = getMetrics(test_confusion_matrix, 0)
itest_precision, itest_recall, itest_dice = getMetrics(test_confusion_matrix, 1)
wtest_precision, wtest_recall, wtest_dice = getMetrics(test_confusion_matrix, 2)

print('TEST :\n\n')
print("CLASS\t\tPRECISION\t\tRECALL\t\t\tDICE")
print('Strong\t\t' + str(stest_precision) + '\t' + str(stest_recall) + '\t' + str(stest_dice))
print('Intermediate\t' + str(itest_precision) +  '\t' + str(itest_recall) + '\t' + str(itest_dice))
print('Weak\t\t' + str(wtest_precision) +   '\t' + str(wtest_recall) + '\t' + str(wtest_dice))

TEST :


CLASS		PRECISION		RECALL			DICE
Strong		0.6748681898066784	0.9770992366412213	0.7935332336842523
Intermediate	0.7850162866449512	0.8281786941580757	0.8010544276631549
Weak		0.528169014084507	0.7537688442211056	0.6163103662307937
